**Подключение нужных библиотек, определение класса Text и чтение данных с файла**

In [12]:
import gzip
from dataclasses import dataclass
from datetime import date
from typing import Optional, Iterator

from yargy import Parser,  or_, and_, rule
from yargy.predicates import gram, is_capitalized, gte, lte, caseless, normalized, dictionary
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("data/news.txt.gz"))

**Извлечение нужных по задаче данных и запись найденных данных в список**

In [13]:
gnc = gnc_relation()

''' Сущность '''
Entry = fact(
    "Entry",
    ["name", "birth_date", "birth_place"]
)

''' Правила для имени/фамилии '''
Name = fact(
    'Name',
    ['first', 'last']
)


FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

LAST = gram('Surn').interpretation(
    Name.last.inflected()
).match(gnc)


NAME = or_(
    rule(
        FIRST,
        LAST
    ),
    rule(
        LAST,
        FIRST
    )

).interpretation(
    Name
)


''' Правила для дат '''
Birth_date = fact(
    'Birth_date',
    ['year', 'month', 'day']
)
class Birth_date(Birth_date):
    @property
    def as_datetime(self):
        return date(self.year, self.month, self.day)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'мая': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}


DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Birth_date.day.custom(int)
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Birth_date.month.custom(int)
)
YEAR = and_(
    gte(1900),
    lte(2024)
).interpretation(
    Birth_date.year.custom(int)
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Birth_date.month.normalized().custom(MONTHS.__getitem__)
)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
BIRTH_DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    ),
    rule(
        YEAR
    )
).interpretation(Birth_date)


''' Правила для места рождения '''
BIRTH_PLACE = or_(
        rule(
            gram("NOUN")
            ),
        rule(
            gram("ADJF"),
            dictionary({
                'столице',
                'городе',
                'селе'
        })
        )
)

ENTRY = or_(
    rule(
       NAME.interpretation(
            Entry.name
        ) 
    ),
    rule(
       NAME.interpretation(
            Entry.name
        ),
        normalized('родиться'),
        gram("PREP"),
        BIRTH_DATE.interpretation(
            Entry.birth_date
        ),
        YEAR_WORDS,
        gram("PREP"),
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        )
    ),
    rule(
       NAME.interpretation(
            Entry.name
        ),
        normalized('родиться'),
        gram("PREP"),
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        ),
        gram("PREP"),
        BIRTH_DATE.interpretation(
            Entry.birth_date
        ), 
    )
).interpretation(
    Entry
)

''' Получение данных и запись их в список '''
res = []
parser = Parser(ENTRY)
for i in texts:
    for match in parser.findall(i.text):
        res.append(match.fact)

**Запись найденных данных в файл**

In [14]:
with open('output_task_1.txt', 'w+', encoding='utf-8') as f:
    for i in res:
        f.write(str(i))
        f.write('\n')